In [1]:
import aliases

In [2]:
from everest.h5anchor import Reader

In [3]:
%%time
reader = Reader('allout', aliases.datadir, 1)
cut = reader['*/.type']
transform = cut == 'Viscoplastic'
selection = reader[transform]
final = selection['*/.H']
final.read()

CPU times: user 29.3 s, sys: 1.62 s, total: 30.9 s
Wall time: 30.9 s


(('/aabaesweuh-luubrbriups', 1.778),
 ('/aabeowauj-psauthkrousn', 10.0),
 ('/aabiaspiot-siangiasto', 0.01),
 ('/aabichebr-iasteswosm', 5.623),
 ('/aablaieakwee-kaekbotr', 0.01),
 ('/aablatzuzh-thuarhtseav', 3.1622776601683795),
 ('/aablauoal-dwaokrieseu', 1.0),
 ('/aablauuilaa-bruobkiutr', 1.0),
 ('/aabliakoadw-uolaoeobii', 0.316),
 ('/aabliapeetr-choaprzhesl', 1.778),
 ('/aabliiauvuu-eerhaflion', 0.316),
 ('/aabliuchuuph-iashiiieblue', 0.01),
 ('/aabliuoikwou-luupsweun', 0.562),
 ('/aabloauuswai-stuelthoogr', 0.032),
 ('/aabloighut-griitrlaodr', 1.778),
 ('/aabloogau-sleaswuothuo', 10.0),
 ('/aabloouihoo-thoatgoowr', 0.056),
 ('/aabluekluph-wrewrakaa', 0.01),
 ('/aablueuusnau-sciatwuichia', 3.162),
 ('/aablugreb-ijaeayai', 0.0),
 ('/aabluiouphue-shufriekri', 0.562),
 ('/aabluoeekloo-iuscirhof', 0.01),
 ('/aablutioh-spaureowreu', 3.162),
 ('/aaboibrousk-skaospchuagh', 0.31622776601683794),
 ('/aabooceog-uibeekliach', 10.0),
 ('/aabreegluag-smoamtzoer', 0.1),
 ('/aabreilefl-ueklaiaqee',

In [ ]:
%%time
reader = Reader('allout', aliases.datadir, 1)
cut = reader['*/.type']
transform = cut == 'Viscoplastic'
selection = reader[transform]
final = selection['*/.H']

In [ ]:
%%time
_ = reader.basekeys, reader.manifest, reader.basekeydict

In [ ]:
%%time
_ = cut.basekeys, cut.manifest, cut.basekeydict

In [ ]:
%%time
_ = transform.basekeys, transform.manifest, transform.basekeydict

In [ ]:
%%time
_ = transform.read()

In [ ]:
%%time
_ = selection.basekeys, selection.manifest, selection.basekeydict

In [ ]:
%%time
_ = final.basekeys, final.manifest, final.basekeydict

In [ ]:
%%time
final.read()

In [ ]:
# %%time
# allnus = cut['*/#Nu'].data